In [57]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen

#AWS Test

In [58]:
URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [59]:
# Big 5 European Leagues (Spain, England, Germany, France, Italy)

big_5_leagues = []

for j in soup.find_all('tbody')[2].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        big_5_leagues.append(j.find('a')['href'])

big_5_leagues = big_5_leagues[:-1]

big_5_leagues

['/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/11/history/Serie-A-Seasons']

In [60]:
# function to obtain league/season URLs

def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [61]:
# All Seasons Big 5 Leagues

all_seasons_big_5 = []

for i in big_5_leagues:
    league_seasons = get_all_seasons(i)
    all_seasons_big_5 += league_seasons

all_seasons_big_5

['/en/comps/9/Premier-League-Stats',
 '/en/comps/9/10728/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/2004-2005-Premier-League-Stats',
 '/en/comps/9/112/2003-2004-Premier-League-Stats',
 '/en/comps/9/84/2002-2003-Premier-Lea

In [62]:
# print(len(all_seasons))
print(len(all_seasons_big_5))

159


In [63]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [64]:
# Getting final_stats for all players in the Big 5 Leagues

final_stats_big_5 = get_all_player_urls(all_seasons_big_5)

In [65]:
final_stats_big_5

['/en/comps/9/stats/Premier-League-Stats',
 '/en/comps/9/10728/stats/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/stats/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/stats/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/stats/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/stats/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/stats/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/stats/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/stats/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/stats/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/stats/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/stats/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/stats/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/stats/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/stats/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/stats/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/stats/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/stats/2004-2005-Prem

In [66]:
def get_player_urls(url_list):
    
    player_url_list = []

    for url in url_list:
        url = 'https://fbref.com/' + url
        res = requests.get(url)
        ## The next two lines get around the issue with comments breaking the parsing.
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",res.text),'lxml')
        all_tables = soup.findAll("tbody")
        try:
            player_table = all_tables[2]
        except:
            next

        for row in player_table.find_all('a'):
            if "matchlogs" in row['href']:
                player_url_list.append(row['href'])

    return player_url_list

In [67]:
#Getting player tables for big 5 leagues

player_table_big_5 = get_player_urls(final_stats_big_5)

len(player_table_big_5)

81277

In [68]:
# unique URLs

len(list(set(player_table_big_5)))

78959

In [69]:
player_table_big_5

['/en/players/774cf58b/matchlogs/2021-2022/summary/Max-Aarons-Match-Logs',
 '/en/players/f2bf1b0f/matchlogs/2021-2022/summary/Che-Adams-Match-Logs',
 '/en/players/9b398aea/matchlogs/2021-2022/summary/Rayan-Ait-Nouri-Match-Logs',
 '/en/players/a8c0acb7/matchlogs/2021-2022/summary/Kristoffer-Ajer-Match-Logs',
 '/en/players/eaeca114/matchlogs/2021-2022/summary/Nathan-Ake-Match-Logs',
 '/en/players/b827d5b3/matchlogs/2021-2022/summary/Marc-Albrighton-Match-Logs',
 '/en/players/77e84962/matchlogs/2021-2022/summary/Thiago-Alcantara-Match-Logs',
 '/en/players/cd1acf9d/matchlogs/2021-2022/summary/Trent-Alexander-Arnold-Match-Logs',
 '/en/players/7a2e46a8/matchlogs/2021-2022/summary/Alisson-Match-Logs',
 '/en/players/cc700722/matchlogs/2021-2022/summary/Allan-Match-Logs',
 '/en/players/cea4ee8f/matchlogs/2021-2022/summary/Dele-Alli-Match-Logs',
 '/en/players/862a1c15/matchlogs/2021-2022/summary/Miguel-Almiron-Match-Logs',
 '/en/players/f4290206/matchlogs/2021-2022/summary/Marcos-Alonso-Match-Lo

In [70]:
# Pull all player stats for all competitions

def get_players_all_competitions(player_list):
    
    player_urls = []

    for i in player_list:
        player_urls.append('https://fbref.com/en/players/' + i.split('/')[3:4][0] + '/all_comps/' 
                            + i.split('/')[7:][0].replace("-Match-Logs", "") + '/-Stats---All-Competitions')

    return list(set(player_urls))

player_all_competitions = get_players_all_competitions(player_table_big_5)
len(player_all_competitions)


19572

In [71]:
player_all_competitions

['https://fbref.com/en/players/94a468e6/all_comps/Arnaud-Buhler/-Stats---All-Competitions',
 'https://fbref.com/en/players/8af2e736/all_comps/Youssef-Salimi/-Stats---All-Competitions',
 'https://fbref.com/en/players/bd20fc57/all_comps/Bernard-Hereson/-Stats---All-Competitions',
 'https://fbref.com/en/players/53070b63/all_comps/Paulinho/-Stats---All-Competitions',
 'https://fbref.com/en/players/ed134a9f/all_comps/Angel-Cuellar/-Stats---All-Competitions',
 'https://fbref.com/en/players/56e4313b/all_comps/Stephan-Beckenbauer/-Stats---All-Competitions',
 'https://fbref.com/en/players/7bcb55da/all_comps/Michel-Boerebach/-Stats---All-Competitions',
 'https://fbref.com/en/players/654f4e63/all_comps/Zinedine-Zidane/-Stats---All-Competitions',
 'https://fbref.com/en/players/79fa1799/all_comps/Karim-Guede/-Stats---All-Competitions',
 'https://fbref.com/en/players/5dfc6ad5/all_comps/Axel-Witsel/-Stats---All-Competitions',
 'https://fbref.com/en/players/6085a079/all_comps/Marco-Pascolo/-Stats---Al

In [72]:
# test if a player exists 

# substring_in_list = any('dea698d9' in string for string in player_all_competitions)                        
# substring_in_list       


In [73]:
# Generate the match log urls for all players across all 

def get_player_match_logs(player_list_summary, line):
    
    res = requests.get(player_list_summary[line])
    soup = BeautifulSoup(res.text,'lxml')

    match_logs_list = []

    for i in soup.find_all('tbody'):
        for j in i.find_all('td', {'data-stat':'matches'}):
            if j.find('a') != None:
                if 'summary' in j.find('a')['href']:
                    match_logs_list.append(j.find('a')['href'])
                    
    return list(set(match_logs_list))

In [74]:
player_all_competitions_df = pd.DataFrame(player_all_competitions)

player_all_competitions_df['FBRefID'] = player_all_competitions_df[0].str.split('/').str[5]

In [75]:
fbref_to_tm_mapping = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')
fbref_to_tm_mapping['FBRefID'] = fbref_to_tm_mapping['UrlFBref'].str.split('/').str[5]
fbref_to_tm_mapping['TMID'] = fbref_to_tm_mapping['UrlTmarkt'].str.split('/').str[6]

In [76]:
fbref_to_tm_mapping

,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back,4f974391,92571
2,AarÃ³n Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper,67669ce7,284430
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield,565c3fe4,276566
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back,1780bb4a,591949
...,...,...,...,...,...,...
6295,Zinho Vanheusden,https://fbref.com/en/players/afbf187b/Zinho-Va...,https://www.transfermarkt.com/zinho-vanheusden...,Centre-Back,afbf187b,338672
6296,Zlatan IbrahimoviÄ,https://fbref.com/en/players/4cde5509/Zlatan-I...,https://www.transfermarkt.com/zlatan-ibrahimov...,Centre-Forward,4cde5509,3455
6297,Zlatko JunuzoviÄ,https://fbref.com/en/players/dba67bc4/Zlatko-J...,https://www.transfermarkt.com/zlatko-junuzovic...,Central Midfield,dba67bc4,31007
6298,Zouhair Feddal,https://fbref.com/en/players/89151b16/Zouhair-...,https://www.transfermarkt.com/zouhair-feddal/p...,Centre-Back,89151b16,129386


In [77]:
player_all_competitions_filtered = player_all_competitions_df.merge(fbref_to_tm_mapping, left_on='FBRefID', right_on='FBRefID', how='inner')

In [80]:
player_all_competitions_filtered[player_all_competitions_filtered['FBRefID'] == 'dea698d9']


player_all_competitions_filtered_list = list(player_all_competitions_filtered[0])

In [81]:
len(player_all_competitions_filtered)

5193

In [86]:

# Total length of player_all_competitions is 5192


match_logs_list = []

# 1st batch 0:5000 - DONE
count = 0
for i in range(len(player_all_competitions_filtered_list)):
    match_logs_list.extend(get_player_match_logs(player_all_competitions_filtered_list, i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions_filtered_list)*100)))
    sys.stdout.flush()

In [ ]:
# export match_logs_list_urls

pd.DataFrame(match_logs_list).to_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/match_logs_list_urls.csv')

In [98]:
match_logs_list_urls = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/match_logs_list_urls.csv')